In [7]:
import os
from openai import OpenAI
import json
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)


In [8]:
def get_completion(messages, model="gpt-4o", tools=[], tool_choice=None):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response.choices[0].message
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [9]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather = {
        "location": location,
        "temperature": 72,
        "unit": unit,
    }

    return json.dumps(weather)

In [10]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string", 
                        "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },   
    }
]

In [11]:
messages = [
    {"role": "user", "content": "What's the weather like in San Francisco?"},
]

In [12]:
response = get_completion(messages, tools=tools)
print(response)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_IgTFlMXoKiDVxCA06QBu63qV', function=Function(arguments='{"location":"San Francisco, CA"}', name='get_current_weather'), type='function')])


In [15]:
args = json.loads(response.tool_calls[0].function.arguments)
args

{'location': 'San Francisco, CA'}

In [16]:
get_current_weather(**args)

'{"location": "San Francisco, CA", "temperature": 72, "unit": "fahrenheit"}'

In [17]:
messages = [
    {
        "role": "user",
        "content": "Hello! How are you?",
    }
]

In [18]:
get_completion(messages, tools=tools)

ChatCompletionMessage(content="Hello! I'm doing great, thank you for asking. How can I assist you today?", role='assistant', function_call=None, tool_calls=None)

In [20]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in London?",
    }
]
get_completion(messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_current_weather"}})

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_vgPNXmA9DEUO2NFZpdwz1xai', function=Function(arguments='{"location":"London, UK","unit":"celsius"}', name='get_current_weather'), type='function')])

In [21]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in London and Belmopan in the coming days?",
    }
]
get_completion(messages, tools=tools)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_fOFghjLDfh4NNrwPZK4xg2rm', function=Function(arguments='{"location": "London"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_Gznn6NrFK0giR46n6vSi5EKZ', function=Function(arguments='{"location": "Belmopan"}', name='get_current_weather'), type='function')])

In [22]:
messages = []
messages.append({"role": "user", "content": "What's the weather like in Boston!"})
assistant_message = get_completion(messages, tools=tools, tool_choice="auto")
assistant_message = json.loads(assistant_message.model_dump_json())
assistant_message["content"] = str(assistant_message["tool_calls"][0]["function"])

#a temporary patch but this should be handled differently
# remove "function_call" from assistant message
del assistant_message["function_call"]

In [23]:
messages.append(assistant_message)

In [24]:
messages

[{'role': 'user', 'content': "What's the weather like in Boston!"},
 {'content': '{\'arguments\': \'{"location":"Boston, MA"}\', \'name\': \'get_current_weather\'}',
  'role': 'assistant',
  'tool_calls': [{'id': 'call_WU6lGHNEgLyIlyMemIU7O3dH',
    'function': {'arguments': '{"location":"Boston, MA"}',
     'name': 'get_current_weather'},
    'type': 'function'}]}]

In [25]:
# get the weather information to pass back to the model
weather = get_current_weather(messages[1]["tool_calls"][0]["function"]["arguments"])

messages.append({"role": "tool",
                 "tool_call_id": assistant_message["tool_calls"][0]["id"],
                 "name": assistant_message["tool_calls"][0]["function"]["name"],
                 "content": weather})

In [26]:
messages

[{'role': 'user', 'content': "What's the weather like in Boston!"},
 {'content': '{\'arguments\': \'{"location":"Boston, MA"}\', \'name\': \'get_current_weather\'}',
  'role': 'assistant',
  'tool_calls': [{'id': 'call_WU6lGHNEgLyIlyMemIU7O3dH',
    'function': {'arguments': '{"location":"Boston, MA"}',
     'name': 'get_current_weather'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_WU6lGHNEgLyIlyMemIU7O3dH',
  'name': 'get_current_weather',
  'content': '{"location": "{\\"location\\":\\"Boston, MA\\"}", "temperature": 72, "unit": "fahrenheit"}'}]

In [27]:
final_response = get_completion(messages, tools=tools)

In [28]:
final_response

ChatCompletionMessage(content='The current temperature in Boston, MA is 72°F. If you need more details or any other information, feel free to ask!', role='assistant', function_call=None, tool_calls=None)